In [158]:
#get data on ways fighter won/lost through dec, won/lost through sub, won/lost through ko/tko  
#tabulate outcomes in a table such as: Outcome, % chance, winner 
#FRONT END: give the user the ability to input two fighters (drop down menu)
# ---- FINISHED --- 
#extra: prevent users from picking two fighters that are from different weight classes
#--- LONG RUN --- 
# Improve model where it takes # of strikes (successful, attempted) , ground game/submissions duration, takedowns (successful, attempted)
# into consideration 

In [159]:
import pandas as pd 

In [160]:
df = pd.read_csv("UFC Matches April 2023.csv")

In [161]:
df.head()

,Full Name,Weight,Matchup,Outcome,Method,Round,Time
0,Tom Aaron,WEIGHT: 155 lbs.,NaN,NaN,NaN,NaN,NaN
1,Tom Aaron,WEIGHT: 155 lbs.,Tom Aaron\n \n Matt Ricehouse,LOSS,U-DEC,3.0,5:00
2,Tom Aaron,WEIGHT: 155 lbs.,Tom Aaron\n \n Eric Steenberg,WIN,SUB,1.0,0:56
3,Danny Abbadi,WEIGHT: 155 lbs.,NaN,NaN,NaN,NaN,NaN
4,Danny Abbadi,WEIGHT: 155 lbs.,Danny Abbadi\n \n Jorge Gurgel,LOSS,S-DEC,3.0,5:00


In [162]:
df.loc[(df["Full Name"] == "Bruno Silva") & (df["Weight"] == "WEIGHT: 185 lbs.")]

,Full Name,Weight,Matchup,Outcome,Method,Round,Time
19225,Bruno Silva,WEIGHT: 185 lbs.,NaN,NaN,NaN,NaN,NaN
19226,Bruno Silva,WEIGHT: 185 lbs.,Bruno Silva\n\nBrad Tavares,NEXT,NaN,NaN,NaN
19227,Bruno Silva,WEIGHT: 185 lbs.,Bruno Silva\n\nGerald Meerschaert,LOSS,SUB,3.0,1:39
19228,Bruno Silva,WEIGHT: 185 lbs.,Bruno Silva\n\nAlex Pereira,LOSS,U-DEC,3.0,5:00
19229,Bruno Silva,WEIGHT: 185 lbs.,Bruno Silva\n\nJordan Wright,WIN,KO/TKO,1.0,1:28
19230,Bruno Silva,WEIGHT: 185 lbs.,Bruno Silva\n\nAndrew Sanchez,WIN,KO/TKO,3.0,2:35
19231,Bruno Silva,WEIGHT: 185 lbs.,Bruno Silva\n\nWellington Turman,WIN,KO/TKO,1.0,4:45


## Data Cleaning

We have to clean our dataset then group each fighter and get their distributions to determine their likelihood of winning/losing via decision, tko/ko or submission.

In [163]:
# delete first row of every fighter
nulls = df[["Matchup", "Outcome", 'Method', 'Round', 'Time']].isnull().all(axis=1) 
df = df.loc[~nulls, :]

#find matches that are upcoming and delete them
df = df[df["Outcome"] != 'NEXT']

# delete "weight text" in weight column
df['Weight'] = df['Weight'].str.replace('WEIGHT: ', '')

# keep only opponents name in matchup column
def remove_name(orig_matchup):
    last_index = orig_matchup.rfind('\n')
    new_string = orig_matchup[last_index +1:]
    return new_string.strip()

df['Matchup'] = df['Matchup'].apply(lambda x: remove_name(x))

#discovered there is a duplicate fighter name 
df.loc[[19227, 19228, 19229, 19230, 19231], 'Full Name'] = "Bruno Blindado Silva"

In [164]:
unique_methods = df['Method'].unique()

In [165]:
unique_methods

array(['U-DEC', 'SUB', 'S-DEC', 'KO/TKO', 'Overturned', 'Other', 'M-DEC',
       'CNC', 'DQ', 'SUB ', 'KO/TKO ', 'S-DEC ', 'Decision', 'U-DEC ',
       'M-DEC ', 'Overturned ', 'CNC '], dtype=object)

In [166]:
unique_outcomes = df['Outcome'].unique()

In [167]:
unique_outcomes

array(['LOSS', 'WIN', 'NC', 'DRAW'], dtype=object)

In [168]:
removed_outcomes = ['NC', 'DRAW']
removed_methods = ['Overturned', 'Other', 'CNC', 'DQ', 'Overturned ', 'CNC ']

In [169]:
filtered_df1 = df.loc[~df['Method'].isin(removed_methods)]
filtered_df2 = filtered_df1.loc[~df['Outcome'].isin(removed_outcomes)]
filtered_df2['Method'] = filtered_df2['Method'].apply(lambda x: x.strip())

<ipython-input-169-fcbd382802ee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df2['Method'] = filtered_df2['Method'].apply(lambda x: x.strip())


In [170]:
df = filtered_df2

## Exploratory Data Analysis (EDA)

In [171]:
decisions = ['U-DEC', 'S-DEC', 'M-DEC', 'Decision']
kos = ['KO/TKO']
submissions = ['SUB']

In [172]:
decision_df = df[df['Method'].isin(decisions)]
decision_count = decision_df['Method'].count()

ko_df = df[df['Method'].isin(kos)]
ko_count = ko_df['Method'].count()

submission_df = df[df['Method'].isin(submissions)]
submission_count = submission_df['Method'].count()

print(decision_count/len(df))
print(ko_count/len(df))
print(submission_count/len(df))

0.428926871484206
0.34714409346603203
0.223929035049762


In [173]:
#make multipliers?
#0.4286023366508005 --> 43% end by decision
#0.34714409346603203 --> 35% end by ko
#0.223929035049762 --> 22% end by submission

In [174]:
#questions that could be of importance:
#what is the most common method of winning?
#for fighters that win through ko/submission what round does it typically occur in?
#is there a discrepancy in the way fighters win across different weight classes?

In [175]:
def transform_dataset(df):
    data = {
    'Total Fights': df.groupby(['Full Name'])['Matchup'].count(),
    'Total Wins': df.loc[df['Outcome'] == 'WIN'].groupby(['Full Name'])['Outcome'].count(),
    'Total Losses': df.loc[df['Outcome'] == 'LOSS'].groupby(['Full Name'])['Outcome'].count(),
    'Wins By Decision': df.loc[(df['Outcome'] == 'WIN') & (df['Method'].isin(decisions))].groupby(['Full Name'])['Outcome'].count(),
    'Wins By KO': df.loc[(df['Outcome'] == 'WIN') & (df['Method'].isin(kos))].groupby(['Full Name'])['Outcome'].count(),
    'Wins By Submission': df.loc[(df['Outcome'] == 'WIN') & (df['Method'].isin(submissions))].groupby(['Full Name'])['Outcome'].count(),
    'Loss By Decision': df.loc[(df['Outcome'] == 'LOSS') & (df['Method'].isin(decisions))].groupby(['Full Name'])['Outcome'].count(),
    'Loss By KO': df.loc[(df['Outcome'] == 'LOSS') & (df['Method'].isin(kos))].groupby(['Full Name'])['Outcome'].count(),
    'Loss By Submission': df.loc[(df['Outcome'] == 'LOSS') & (df['Method'].isin(submissions))].groupby(['Full Name'])['Outcome'].count()
}
    fighter_stats = pd.DataFrame(data)
    fighter_stats.fillna(0, inplace=True)
    return fighter_stats

In [176]:
df = transform_dataset(df)

In [177]:
df.head()

,Total Fights,Total Wins,Total Losses,Wins By Decision,Wins By KO,Wins By Submission,Loss By Decision,Loss By KO,Loss By Submission
AJ Dobson,3,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0
AJ Fletcher,4,2.0,2.0,0.0,1.0,1.0,2.0,0.0,0.0
AJ Fonseca,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
AJ Matthews,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
Aalon Cruz,3,1.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0


## Monte Carlo Simulation

In [178]:
import math
import random as rnd
import numpy.random as npr

In [179]:
def calculate_mean(statistic, total_fights):
    return statistic/total_fights

#def calculate_sd(statistic, total_fights):
 #   mean = calculate_mean(statistic, total_fights)
  #  return math.sqrt((mean*(1-mean))/total_fights)

def calculate_sd(statistic, total_fights):
    mean = calculate_mean(statistic, total_fights)
    return math.sqrt(mean*(1-mean))

In [180]:
def get_fighter_parameters(fighter1, fighter2):
    data = {"Decision Wins Mean" : [calculate_mean(df.loc[fighter1, "Wins By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Wins By Decision"], df.loc[fighter2, "Total Fights"])], 
            
           "Decision Wins SD" : [calculate_sd(df.loc[fighter1, "Wins By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Wins By Decision"], df.loc[fighter2, "Total Fights"])],
            
           "KO Wins Mean" : [calculate_mean(df.loc[fighter1, "Wins By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Wins By KO"], df.loc[fighter2, "Total Fights"])], 
            
            'KO Wins SD' : [calculate_sd(df.loc[fighter1, "Wins By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Wins By KO"], df.loc[fighter2, "Total Fights"])], 
           
            'Sub Wins Mean' : [calculate_mean(df.loc[fighter1, "Wins By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Wins By Submission"], df.loc[fighter2, "Total Fights"])],
           
            'Sub Wins SD' : [calculate_sd(df.loc[fighter1, "Wins By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Wins By Submission"], df.loc[fighter2, "Total Fights"])], 
            
            'Decision Loss Mean' : [calculate_mean(df.loc[fighter1, "Loss By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Loss By Decision"], df.loc[fighter2, "Total Fights"])], 
            
            'Decision Loss SD' : [calculate_sd(df.loc[fighter1, "Loss By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Loss By Decision"], df.loc[fighter2, "Total Fights"])], 
            
            'KO Loss Mean' : [calculate_mean(df.loc[fighter1, "Loss By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Loss By KO"], df.loc[fighter2, "Total Fights"])], 
            
            'KO Loss SD' : [calculate_sd(df.loc[fighter1, "Loss By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Loss By KO"], df.loc[fighter2, "Total Fights"])], 
           
            'Sub Loss Mean' : [calculate_mean(df.loc[fighter1, "Loss By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Loss By Submission"], df.loc[fighter2, "Total Fights"])], 
            
            'Sub Loss SD' : [calculate_sd(df.loc[fighter1, "Loss By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Loss By Submission"], df.loc[fighter2, "Total Fights"])],
            'Number of Matches' : [df.loc[fighter1, "Total Fights"], df.loc[fighter2, "Total Fights"]]
           
           }
    
    fighter_parameter_df = pd.DataFrame(data=data, index = [fighter1, fighter2])
    return fighter_parameter_df

In [181]:
def gameSim():
    results = []
    fighter1_dec_score = (rnd.gauss(matchup_df.iloc[0]['Decision Wins Mean'],matchup_df.iloc[0]['Decision Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[1]['Decision Loss Mean'],matchup_df.iloc[1]['Decision Loss SD']))/2
    fighter1_ko_score = (rnd.gauss(matchup_df.iloc[0]['KO Wins Mean'],matchup_df.iloc[0]['KO Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[1]['KO Loss Mean'],matchup_df.iloc[1]['KO Loss SD']))/2
    fighter1_sub_score = (rnd.gauss(matchup_df.iloc[0]['Sub Wins Mean'],matchup_df.iloc[0]['Sub Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[1]['Sub Loss Mean'],matchup_df.iloc[1]['Sub Loss SD']))/2
    fighter2_dec_score = (rnd.gauss(matchup_df.iloc[1]['Decision Wins Mean'],matchup_df.iloc[1]['Decision Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[0]['Decision Loss Mean'],matchup_df.iloc[0]['Decision Loss SD']))/2
    fighter2_ko_score = (rnd.gauss(matchup_df.iloc[1]['KO Wins Mean'],matchup_df.iloc[1]['KO Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[0]['KO Loss Mean'],matchup_df.iloc[0]['KO Loss SD']))/2
    fighter2_sub_score = (rnd.gauss(matchup_df.iloc[1]['Sub Wins Mean'],matchup_df.iloc[1]['Sub Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[0]['Sub Loss Mean'],matchup_df.iloc[0]['Sub Loss SD']))/2   
    
    results.append(fighter1_dec_score)
    results.append(fighter1_ko_score)
    results.append(fighter1_sub_score)    
    results.append(fighter2_dec_score)    
    results.append(fighter2_ko_score)    
    results.append(fighter2_sub_score)
    
    
    if max(results) == results[0]:
        return "f1_dec"
    elif max(results) == results[1]:
        return "f1_ko"
    elif max(results) == results[2]:
        return "f1_sub"
    elif max(results) == results[3]:
        return "f2_dec"
    elif max(results) == results[4]:
        return "f2_ko"
    elif max(results) == results[5]:
        return "f2_sub"
    else: return rnd.choice(["f1_dec", "f1_ko", "f1_sub", "f2_dec", "f2_ko", "f2_sub"])

In [182]:
#def gameSim():
 #   results = []
  #  fighter1_dec_score = (npr.binomial(matchup_df.iloc[0]["Number of Matches"], matchup_df.iloc[0]['Decision Wins Mean'])+ 
   #                       npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Decision Loss Mean']))/2
    #fighter1_ko_score = (npr.binomial(matchup_df.iloc[0]["Number of Matches"], matchup_df.iloc[0]['KO Wins Mean'])+ 
   #                       npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['KO Loss Mean']))/2
    #fighter1_sub_score = (npr.binomial(matchup_df.iloc[0]["Number of Matches"], matchup_df.iloc[0]['Sub Wins Mean'])+ 
     #                     npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Sub Loss Mean']))/2
    #fighter2_dec_score = (npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Decision Wins Mean'])+ 
    #                      npr.binomial(matchup_df.iloc[0]['Number of Matches'], matchup_df.iloc[0]['Decision Loss Mean']))/2
    #fighter2_ko_score = (npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['KO Wins Mean'])+ 
     #                     npr.binomial(matchup_df.iloc[0]['Number of Matches'], matchup_df.iloc[0]['KO Loss Mean']))/2
    #fighter2_sub_score = (npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Sub Wins Mean'])+ 
     #                     npr.binomial(matchup_df.iloc[0]['Number of Matches'], matchup_df.iloc[0]['Sub Loss Mean']))/2 
    
    #results.append(fighter1_dec_score)
    #results.append(fighter1_ko_score)
    #results.append(fighter1_sub_score)    
    #results.append(fighter2_dec_score)    
    #results.append(fighter2_ko_score)    
    #results.append(fighter2_sub_score)
    
    
    #if max(results) == results[0]:
     #   return "f1_dec"
    #elif max(results) == results[1]:
     #   return "f1_ko"
    #elif max(results) == results[2]:
     #   return "f1_sub"
    #elif max(results) == results[3]:
     #   return "f2_dec"
    #elif max(results) == results[4]:
     #   return "f2_ko"
    #elif max(results) == results[5]:
     #   return "f2_sub"
    #else: return "tie"

In [183]:
def gamesSim(ns):
    matchesout = []
    fighter1_decwin = 0
    fighter1_kowin = 0
    fighter1_subwin = 0
    fighter2_decwin = 0
    fighter2_kowin = 0
    fighter2_subwin = 0
    tie = 0
    for i in range(ns):
        gm = gameSim()
        matchesout.append(gm)
        if gm == "f1_dec":
            fighter1_decwin +=1 
        elif gm == "f1_ko":
            fighter1_kowin +=1 
        elif gm == "f1_sub":
            fighter1_subwin +=1 
        elif gm == "f2_dec":
            fighter2_decwin +=1 
        elif gm == "f2_ko":
            fighter2_kowin +=1 
        else:
            fighter2_subwin +=1 

    
    print(matchup_df.index[0] +' Decision Win ', round((fighter1_decwin/ns)*100,2),'%')
    print(matchup_df.index[0] +' KO Win ', round((fighter1_kowin/ns)*100,2),'%')
    print(matchup_df.index[0] +' Submission Win ', round((fighter1_subwin/ns)*100,2),'%')
    print(matchup_df.index[1] +' Decision Win ', round((fighter2_decwin/ns)*100,2),'%')
    print(matchup_df.index[1] +' KO Win ', round((fighter2_kowin/ns)*100,2),'%')
    print(matchup_df.index[1] +' Submission Win ', round((fighter2_subwin/ns)*100,2),'%')
    #print('Tie ', (tie/ns)*100, '%')
    
    #return matchesout

## Predictions on UFC Fight Night: Pavlovich vs Blaydes

### Sergei Pavlovich vs Curtis Blaydes

In [184]:
matchup_df = get_fighter_parameters("Sergei Pavlovich", "Curtis Blaydes")
gamesSim(10000)
#sergei ko
#all correct

Sergei Pavlovich Decision Win  0.09 %
Sergei Pavlovich KO Win  62.68 %
Sergei Pavlovich Submission Win  0.0 %
Curtis Blaydes Decision Win  9.05 %
Curtis Blaydes KO Win  28.18 %
Curtis Blaydes Submission Win  0.0 %


### Brad Tavares vs Bruno Silva

In [185]:
matchup_df = get_fighter_parameters("Brad Tavares", "Bruno Blindado Silva")
gamesSim(10000)
#bruno silva ko
#all wrong
#2nd highest option correct

Brad Tavares Decision Win  44.46 %
Brad Tavares KO Win  2.15 %
Brad Tavares Submission Win  5.97 %
Bruno Blindado Silva Decision Win  5.62 %
Bruno Blindado Silva KO Win  41.74 %
Bruno Blindado Silva Submission Win  0.06 %


### Bobby Green vs Jared Gordon - No Contest

In [186]:
matchup_df = get_fighter_parameters("Bobby Green", "Jared Gordon")
gamesSim(10000)

Bobby Green Decision Win  22.04 %
Bobby Green KO Win  15.19 %
Bobby Green Submission Win  6.99 %
Jared Gordon Decision Win  47.9 %
Jared Gordon KO Win  7.35 %
Jared Gordon Submission Win  0.53 %


### Iasmin Lucindo vs Brogan Walker

In [187]:
matchup_df = get_fighter_parameters("Iasmin Lucindo", "Brogan Walker")
gamesSim(10000)
#iasmin decision
#winner correct, method wrong

Iasmin Lucindo Decision Win  0.0 %
Iasmin Lucindo KO Win  100.0 %
Iasmin Lucindo Submission Win  0.0 %
Brogan Walker Decision Win  0.0 %
Brogan Walker KO Win  0.0 %
Brogan Walker Submission Win  0.0 %


### Jeremiah Wells vs Matthew Semelsberger

In [188]:
matchup_df = get_fighter_parameters("Jeremiah Wells", "Matthew Semelsberger")
gamesSim(10000)
#jeremiah decision
#winner correct, method wrong

Jeremiah Wells Decision Win  11.91 %
Jeremiah Wells KO Win  39.93 %
Jeremiah Wells Submission Win  15.05 %
Matthew Semelsberger Decision Win  21.37 %
Matthew Semelsberger KO Win  11.72 %
Matthew Semelsberger Submission Win  0.02 %


### Rick Glenn vs Christos Giagos

In [189]:
matchup_df = get_fighter_parameters("Ricky Glenn", "Christos Giagos")
gamesSim(10000)
#christos ko
#winner correct, method wrong

Ricky Glenn Decision Win  33.85 %
Ricky Glenn KO Win  9.21 %
Ricky Glenn Submission Win  9.62 %
Christos Giagos Decision Win  41.48 %
Christos Giagos KO Win  0.09 %
Christos Giagos Submission Win  5.75 %


### Rani Yahya vs Montel Jackson

In [190]:
matchup_df = get_fighter_parameters("Rani Yahya", "Montel Jackson")
gamesSim(10000)
#montel ko 
#winner correct, method wrong

Rani Yahya Decision Win  24.9 %
Rani Yahya KO Win  0.12 %
Rani Yahya Submission Win  20.51 %
Montel Jackson Decision Win  40.9 %
Montel Jackson KO Win  10.62 %
Montel Jackson Submission Win  2.95 %


### Karol Rosa vs Norma Dumont

In [191]:
matchup_df = get_fighter_parameters("Karol Rosa", "Norma Dumont")
gamesSim(10000)
#norma decision
#all wrong
#2nd highest option was correct

Karol Rosa Decision Win  55.48 %
Karol Rosa KO Win  3.38 %
Karol Rosa Submission Win  0.09 %
Norma Dumont Decision Win  41.05 %
Norma Dumont KO Win  0.0 %
Norma Dumont Submission Win  0.0 %


### Mohammed Usman vs Junior Tafa - N/A (Junior Tafa's first fight recorded on fightmetric)

### Francis Marshall vs William Gomis

In [192]:
matchup_df = get_fighter_parameters("Francis Marshall", "William Gomis")
gamesSim(10000)
#william decision
#all correct

Francis Marshall Decision Win  14.54 %
Francis Marshall KO Win  14.38 %
Francis Marshall Submission Win  0.0 %
William Gomis Decision Win  71.08 %
William Gomis KO Win  0.0 %
William Gomis Submission Win  0.0 %


### Brady Hiestand vs Batgerel Danaa

In [193]:
matchup_df = get_fighter_parameters("Brady Hiestand", "Batgerel Danaa")
gamesSim(10000)
#brady ko
#winner correct, method wrong

Brady Hiestand Decision Win  51.41 %
Brady Hiestand KO Win  5.15 %
Brady Hiestand Submission Win  0.07 %
Batgerel Danaa Decision Win  22.23 %
Batgerel Danaa KO Win  21.14 %
Batgerel Danaa Submission Win  0.0 %


In [194]:
#2/9 perfect
#4/9 if 2nd highest outcome is accepted
#7/9 winner correct

## Predictions on UFC Fight Night: Song vs Simón

### Song Yadong vs Ricky Simón

In [195]:
matchup_df = get_fighter_parameters("Song Yadong", "Ricky Simon")
gamesSim(10000)
#song ko
#all wrong
#2nd option was correct 

Song Yadong Decision Win  20.0 %
Song Yadong KO Win  27.33 %
Song Yadong Submission Win  2.28 %
Ricky Simon Decision Win  31.56 %
Ricky Simon KO Win  7.48 %
Ricky Simon Submission Win  11.35 %


### Caio Borralho vs Michal Oleksiejczuk

In [196]:
matchup_df = get_fighter_parameters("Caio Borralho", "Michal Oleksiejczuk")
gamesSim(10000)
#caio submission
#winner correct, method wrong 

Caio Borralho Decision Win  59.58 %
Caio Borralho KO Win  6.12 %
Caio Borralho Submission Win  7.23 %
Michal Oleksiejczuk Decision Win  2.19 %
Michal Oleksiejczuk KO Win  24.87 %
Michal Oleksiejczuk Submission Win  0.01 %


### Rodolfo Vieira vs Cody Brundage

In [197]:
matchup_df = get_fighter_parameters("Rodolfo Vieira", "Cody Brundage")
gamesSim(10000)
#rodolfo submission
#all correct

Rodolfo Vieira Decision Win  7.25 %
Rodolfo Vieira KO Win  19.11 %
Rodolfo Vieira Submission Win  35.37 %
Cody Brundage Decision Win  7.36 %
Cody Brundage KO Win  7.59 %
Cody Brundage Submission Win  23.32 %


### Julian Erosa vs Fernando Padilla - N/A (Fernando Padilla's first fight recorded on fightmetric)

### Marcos Rogério de Lima vs Waldo Cortes-Acosta

In [198]:
matchup_df = get_fighter_parameters("Marcos Rogerio de Lima", "Cody Brundage")
gamesSim(10000)
#marcos decision
#winner correct, method wrong

Marcos Rogerio de Lima Decision Win  15.48 %
Marcos Rogerio de Lima KO Win  42.64 %
Marcos Rogerio de Lima Submission Win  3.28 %
Cody Brundage Decision Win  3.5 %
Cody Brundage KO Win  6.57 %
Cody Brundage Submission Win  28.53 %


### Josh Quinlan vs Trey Waters

In [199]:
matchup_df = get_fighter_parameters("Josh Quinlan", "Trey Waters")
gamesSim(10000)
#trey decision
#all wrong 

Josh Quinlan Decision Win  0.0 %
Josh Quinlan KO Win  100.0 %
Josh Quinlan Submission Win  0.0 %
Trey Waters Decision Win  0.0 %
Trey Waters KO Win  0.0 %
Trey Waters Submission Win  0.0 %


### Martin Buday vs Jake Collier

In [200]:
matchup_df = get_fighter_parameters("Martin Buday", "Jake Collier")
gamesSim(10000)
#martin decision
#all correct

Martin Buday Decision Win  53.96 %
Martin Buday KO Win  34.07 %
Martin Buday Submission Win  0.04 %
Jake Collier Decision Win  8.23 %
Jake Collier KO Win  1.77 %
Jake Collier Submission Win  1.93 %


### Cody Durden vs Charles Johnson

In [201]:
matchup_df = get_fighter_parameters("Cody Durden", "Charles Johnson")
gamesSim(10000)
#cody decision
#all correct

Cody Durden Decision Win  58.8 %
Cody Durden KO Win  6.99 %
Cody Durden Submission Win  0.06 %
Charles Johnson Decision Win  8.47 %
Charles Johnson KO Win  9.11 %
Charles Johnson Submission Win  16.57 %


In [202]:
#3/7 perfect
#4/7 if 2nd highest outcome is accepted
#5/7 winner correct

### Stephanie Egger vs Irina Alekseeva - N/A (Irina Alekseeva first fight recorded on fightmetric)

### Journey Newson vs Marcus McGhee - N/A (Marcus McGhee first fight recorded on fightmetric)

### Hailey Cowan vs Jamey-Lyn Horth - N/A (Jamey-Lyn Horth first fight recorded on fightmetric)

## Predictions on UFC 288: Sterling vs. Cejudo

### Aljamain Sterling vs Henry Cejudo

In [203]:
matchup_df = get_fighter_parameters("Aljamain Sterling", "Henry Cejudo")
gamesSim(10000)

Aljamain Sterling Decision Win  27.77 %
Aljamain Sterling KO Win  7.9 %
Aljamain Sterling Submission Win  9.12 %
Henry Cejudo Decision Win  35.88 %
Henry Cejudo KO Win  19.31 %
Henry Cejudo Submission Win  0.02 %


### Belal Muhammad vs Gilbert Burns

In [204]:
matchup_df = get_fighter_parameters("Belal Muhammad", "Gilbert Burns")
gamesSim(10000)

Belal Muhammad Decision Win  48.66 %
Belal Muhammad KO Win  8.81 %
Belal Muhammad Submission Win  0.89 %
Gilbert Burns Decision Win  24.92 %
Gilbert Burns KO Win  8.06 %
Gilbert Burns Submission Win  8.66 %


### Jessica Andrade vs Yan Xiaonan

In [205]:
matchup_df = get_fighter_parameters("Jessica Andrade", "Yan Xiaonan")
gamesSim(10000)

Jessica Andrade Decision Win  19.93 %
Jessica Andrade KO Win  13.78 %
Jessica Andrade Submission Win  3.05 %
Yan Xiaonan Decision Win  57.3 %
Yan Xiaonan KO Win  2.69 %
Yan Xiaonan Submission Win  3.25 %


### Movsar Evloev vs Bryce Mitchell

In [206]:
matchup_df = get_fighter_parameters("Movsar Evloev", "Bryce Mitchell")
gamesSim(10000)

Movsar Evloev Decision Win  72.68 %
Movsar Evloev KO Win  0.0 %
Movsar Evloev Submission Win  0.67 %
Bryce Mitchell Decision Win  26.11 %
Bryce Mitchell KO Win  0.0 %
Bryce Mitchell Submission Win  0.54 %


### Kron Gracie vs Charles Jourdain

In [207]:
matchup_df = get_fighter_parameters("Kron Gracie", "Charles Jourdain")
gamesSim(10000)

Kron Gracie Decision Win  20.56 %
Kron Gracie KO Win  0.11 %
Kron Gracie Submission Win  34.95 %
Charles Jourdain Decision Win  33.71 %
Charles Jourdain KO Win  7.98 %
Charles Jourdain Submission Win  2.69 %


### Drew Dober vs Matt Frevola

In [208]:
matchup_df = get_fighter_parameters("Drew Dober", "Matt Frevola")
gamesSim(10000)

Drew Dober Decision Win  11.97 %
Drew Dober KO Win  42.98 %
Drew Dober Submission Win  0.88 %
Matt Frevola Decision Win  19.7 %
Matt Frevola KO Win  9.47 %
Matt Frevola Submission Win  15.0 %


### Kennedy Nzechukwu vs Devin Clark

In [209]:
matchup_df = get_fighter_parameters("Kennedy Nzechukwu", "Devin Clark")
gamesSim(10000)

Kennedy Nzechukwu Decision Win  10.98 %
Kennedy Nzechukwu KO Win  42.91 %
Kennedy Nzechukwu Submission Win  6.71 %
Devin Clark Decision Win  31.46 %
Devin Clark KO Win  5.87 %
Devin Clark Submission Win  2.07 %


### Khaos Williams vs Rolando Bedoya - N/A (Rolando Bedoya first fight recorded on fightmetric)

### Marina Rodriguez vs Virna Jandiroba

In [210]:
matchup_df = get_fighter_parameters("Marina Rodriguez", "Virna Jandiroba")
gamesSim(10000)

Marina Rodriguez Decision Win  63.64 %
Marina Rodriguez KO Win  6.94 %
Marina Rodriguez Submission Win  0.08 %
Virna Jandiroba Decision Win  9.76 %
Virna Jandiroba KO Win  9.53 %
Virna Jandiroba Submission Win  10.05 %


### Braxton Smith vs Parker Porter - N/A (Braxton Smith first fight recorded on fightmetric)

### Phil Hawes vs Ikram Aliskerov

In [211]:
matchup_df = get_fighter_parameters("Phil Hawes", "Ikram Aliskerov")
gamesSim(10000)

Phil Hawes Decision Win  3.49 %
Phil Hawes KO Win  9.04 %
Phil Hawes Submission Win  0.0 %
Ikram Aliskerov Decision Win  0.0 %
Ikram Aliskerov KO Win  8.77 %
Ikram Aliskerov Submission Win  78.7 %


### Rafael Estevam vs Zhalgas Zhumagulov

In [212]:
matchup_df = get_fighter_parameters("Rafael Estevam", "Zhalgas Zhumagulov")
gamesSim(10000)

Rafael Estevam Decision Win  20.79 %
Rafael Estevam KO Win  77.6 %
Rafael Estevam Submission Win  0.01 %
Zhalgas Zhumagulov Decision Win  0.0 %
Zhalgas Zhumagulov KO Win  0.0 %
Zhalgas Zhumagulov Submission Win  1.6 %


### Joseph Holmes vs Claudio Ribeiro

In [213]:
matchup_df = get_fighter_parameters("Joseph Holmes", "Claudio Ribeiro")
gamesSim(10000)

Joseph Holmes Decision Win  0.07 %
Joseph Holmes KO Win  26.93 %
Joseph Holmes Submission Win  26.24 %
Claudio Ribeiro Decision Win  10.14 %
Claudio Ribeiro KO Win  26.25 %
Claudio Ribeiro Submission Win  10.37 %


### Daniel Santos vs Johnny Munoz

In [214]:
matchup_df = get_fighter_parameters("Daniel Santos", "Johnny Munoz")
gamesSim(10000)

Daniel Santos Decision Win  8.04 %
Daniel Santos KO Win  41.84 %
Daniel Santos Submission Win  0.11 %
Johnny Munoz Decision Win  41.41 %
Johnny Munoz KO Win  0.0 %
Johnny Munoz Submission Win  8.6 %
